# Setup

In [1]:
import numpy as np
import gzip

from astropy.utils.data import download_file
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS

import matplotlib.pyplot as plt

from bliss.utils.download_utils import download_file_to_dst

def plot_image(hdu):
    ax = plt.subplot(projection=WCS(hdu.header))
    ax.imshow(hdu.data,cmap='gray',vmin=hdu.data.min(),vmax=hdu.data.min()+(hdu.data.max()-hdu.data.min())/100.)

URLBASE = "https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9"

ra, dec = 336.635, -0.96

# Basic (RA, Dec) <--> Brick conversion
survey_bricks_filename = download_file(f"{URLBASE}/south/survey-bricks-dr9-south.fits.gz", cache=True, show_progress=True, timeout=120)

# ra1 - lower RA boundary
# ra2 - upper RA boundary
# dec1 - lower Dec boundary
# dec2 - upper Dec boundary
survey_bricks = Table.read(survey_bricks_filename)
brickname = survey_bricks[
            (survey_bricks["ra1"] <= ra)
            & (survey_bricks["ra2"] >= ra)
            & (survey_bricks["dec1"] <= dec)
            & (survey_bricks["dec2"] >= dec)
        ]["brickname"][0]
print(f"Brick for RA, Dec ({ra}, {dec}):", brickname)

Brick for RA, Dec (336.635, -0.96): 3366m010


# DECaLS Original Co-added Images

Get images

In [3]:
image_g_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-g.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)
image_r_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-r.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)
image_z_filename = download_file("{}/south/coadd/{}/{}/legacysurvey-{}-image-z.fits.fz".format(URLBASE, brickname[:3], brickname, brickname), cache=False)

In [4]:
image_g = fits.open(image_g_filename)
image_r = fits.open(image_r_filename)
image_z = fits.open(image_z_filename)

# DECaLS Tractor Catalogs

In [ ]:
tractor_filename = "/home/zhteoh/871-decals-e2e/data/decals/tractor-3366m010.fits"
tractor = Table.read(tractor_filename)

tractor[:5].show_in_notebook()

# Simulate DECaLS Co-Added Images

inverse-variance weighted coadd, units of nanomaggies/pixel

In [ ]:
# Get (some) CCDs for this brick

# b_ccds for b in bands (all unique filters for this brick)

# coadded weight map
# cow    = np.zeros((H,W), np.float32)
# coadded weighted image map
# cowimg = np.zeros((H,W), np.float32)

# for b in bands:
#    for ccd in b_ccds:
#        cowimg
#        # surface-brightness correction
#        cowimg.sbscale = (targetwcs.pixel_scale() / ccd image wcs.pixel_scale()) ** 2
#        # align CCD to target wcs => get output Y, X coords; and corresponding input Y, X coords
#        Yo, Xo, Yi, Xi = align_wcs(ccd.wcs, targetwcs, co_img)
#        
#        im = ccd[Yi, Xi]
#        iv = ccd.invvar[Yi, Xi]
#        # apply surface-brightness scaling (if present)
#        if cowimg.sbscale:
#           im *= cowimg.sbscale
#           iv /= (cowimg.sbscale**2)
#        cowimg[Yo, Xo] += im * iv
#        cow   [Yo, Xo] += iv
#        
#        # per-band normalization
#        tinyw = 1e-30
#        cowimg /= np.maximum(cow, 1e-30)

Get CCDs used for this brick

In [2]:
brick_ccds_filename = download_file(f"{URLBASE}/south/coadd/{brickname[:3]}/{brickname}/legacysurvey-{brickname}-ccds.fits", cache=False, show_progress=True, timeout=120)
brick_ccds = Table.read(brick_ccds_filename)
print("legacysurvey-{brick}-ccds columns: ", brick_ccds.colnames)

# Get unique expnums for each band
# uniformly sample one exposure per band
all_expnums = {}
expnums = {}
for b in ["g", "r", "z"]:
    all_expnums[b] = np.unique(brick_ccds["expnum"][brick_ccds["filter"] == b])
    expnums[b] = np.random.choice(all_expnums[b])

print("uniformly sampled expnums:", expnums)

# filter ccds to only include the selected exposures
brick_ccds_filtered = brick_ccds[np.isin(brick_ccds["expnum"], list(expnums.values()))]

NameError: name 'download_file' is not defined

In [3]:
from bliss.surveys.decals import DECaLS
from bliss.surveys.des import DES

DECaLS.create_des_objs("/data/scratch/zhteoh/871-decals-e2e/data/decals", {brickname: row0})
des_obj, idx = DECaLS.des_obj(
    brickname,
    {"ra": row0["ra"], "dec": row0["dec"]},
    row0["ccdname"],
    row0["filter"],
    row0["image_filename"].split(".fits.fz")[0],
)
des_img = des_obj["image"]
image_h, image_w = des_img.shape
px, py = image_w // 2, image_h // 2
psf_img = DECaLS.des_objs.psf.get_psf_via_despsfex(
    des_image_id=DECaLS.des_objs[idx], px=px, py=py
)[DES.BANDS.index(brick_ccds["filter"])]

ValueError: need at least one array to stack

Inspect depth of brick band images

In [18]:
image_g[1].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / data type of original image                    
NAXIS   =                    2 / dimension of original image                    
NAXIS1  =                 3600 / length of original image axis                  
NAXIS2  =                 3600 / length of original image axis                  
ZDITHER0=                 6159 / dithering offset when quantizing floats        
COMMENT Data product of the DESI Imaging Legacy Surveys                         
COMMENT Full documentation at http://legacysurvey.org                           
LEGPIPEV= 'DR9.6.5 '           / legacypipe git version                         
LSDIR   = '/global/cscratch1/sd/ziyaoz/dr9m/south' / $LEGACY_SURVEY_DIR director
LSDR    = 'DR9     '           / Data release number                            
RUNDATE = '2020-09-13T23:25:01.279042' / runbrick.py run time                   
SURVEY  = 'DECaLS+BASS+MzLS'

Constant sky level (subtracted from co-added image)

In [1]:
sky_g = image_g[1].header["COSKY_G"]
sky_r = image_r[1].header["COSKY_R"]
sky_z = image_z[1].header["COSKY_Z"]

print("(Constant) Sky Levels: ")
print(" g: ", sky_g)
print(" r: ", sky_r)
print(" z: ", sky_z)

NameError: name 'image_g' is not defined

Co-add PSF

In [ ]:
# See https://github.com/legacysurvey/legacypipe/blob/ba1ffd4969c1f920566e780118c542d103cbd9a5/py/legacypipe/coadds.py#L486-L519
# See tractor/psf.py `getPointSourcePatch` (https://github.com/dstndstn/tractor/blob/cdb82000422e85c9c97b134edadff31d68bced0c/tractor/psf.py#L446-L516)

# `tractor/psf.py PixelizedPSF::getPointSourcePatch`:
# let center of psf patch be: px = w//2, py = h//2
# get PSF image at desired pixel location
# psf_img = psf.getImage(px, py)
# H, W = psf_img.shape
# ix = round(float(px))
# iy = round(float(py))
# dx = px - ix
# dy = py - iy
# x0 = ix - W//2
# y0 = iy - H//2

# align psf_img 